In [1]:
import openml
import os
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from category_encoders.wrapper import PolynomialWrapper
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from fastprogress.fastprogress import master_bar, progress_bar

In [2]:
ids = [1021, 838, 1520, 1100, 40683, 1548, 446, 48, 1020, 815, 748, 41082, 1064, 818, 754,
       1012, 1011, 1552, 828, 463, 1025, 1048, 40691, 59, 773, 1117, 41000, 937, 1555, 1004,
       1488, 750, 853, 1535, 1009, 862, 762, 1054, 819, 1559, 841, 1065, 860, 1519, 1530, 1466,
       736, 958, 1005, 14, 903, 40680, 827, 329, 464, 472, 1501, 1055, 763, 789, 40900, 40704,
       30, 942, 907, 902, 679, 23, 867, 931, 731, 40975, 983, 1467, 1014, 40671, 36, 1028, 733, 737,
       785, 887, 886, 1016, 753, 1413, 964, 885, 40687, 915, 40677, 1565, 726, 796, 908, 895, 775,
       926, 1121, 720, 768, 936, 1553, 774, 1554, 1066, 717, 1547, 469, 848, 377, 866, 830,
       43, 375, 814, 1507, 40665, 475, 40497, 40713, 749, 1061, 794, 946, 1506, 1566, 336, 933,
       450, 50, 863, 811, 813, 685, 1460, 682, 40669, 804, 780, 918, 694, 950, 716, 1464, 769, 40678,
       783, 40910, 337, 714, 900, 1462, 1556, 921, 890, 40, 767, 790, 938, 16, 855, 826, 859, 479, 23499,
       1523, 40707, 1499, 1026, 1465, 824, 53, 40983, 807, 836, 1516, 338, 758, 40981, 850, 40997, 741,
       911, 740, 928, 949, 829, 970, 729, 1504, 906, 4153, 888, 346, 745, 797, 759, 730, 468, 1541,
       1539, 1006, 994, 1564, 1524, 910, 778, 1549, 884, 801, 761, 912, 22, 467, 765, 779, 458, 40682, 980,
       851, 1526, 1056, 817, 54, 925, 41007, 18, 40690, 1512, 891, 1496, 837, 870, 1075, 339, 40984, 1558,
       1600, 947, 1049, 732, 777, 1489, 1412, 945, 40686, 743, 803, 721, 1495, 719, 889, 718, 1167, 1538,
       312, 1497, 1551, 40666, 934, 879, 11, 744, 764, 805, 873, 40646, 834, 40709, 746, 1479, 1069,
       40706, 40708, 40496, 971, 782, 3, 857, 909, 31, 965, 1073, 40700, 772, 28, 1115, 41005, 793,
       1050, 1510, 44, 1557, 874, 742, 1475, 1068, 1480, 1071, 723, 847, 995, 40982, 735, 1490,
       40711, 868, 849, 1511, 40693, 40710, 974, 1540, 997, 461, 40705, 920, 40474, 41004, 791, 1508,
       1525, 987, 448, 806, 770, 788, 1498, 1015, 792, 869, 4538, 40994, 37, 905, 991, 1527, 60, 724,
       766, 715, 747, 955, 459, 927, 988, 969, 1494, 1228, 835, 896, 40702, 864, 943, 800, 979, 40663,
       812, 922, 951, 40999, 1463, 941, 892, 40701, 444, 1528, 728, 893, 962, 795, 12, 40660, 1563,
       913, 787, 1487, 1529, 316, 713, 40670, 1455, 1560, 917, 882, 799, 1473, 845, 978, 875, 973, 820]

In [3]:
seed = 42
path = '../data/'
os.makedirs(path, exist_ok=True)

In [ ]:
for dataid in ids:
    dataset = openml.datasets.get_dataset(dataid)
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)
    col_names = attribute_names
    if any(categorical_indicator):
        encoder = PolynomialWrapper(TargetEncoder())
        X = encoder.fit_transform(X, y)
        col_names = X.columns
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    df = pd.DataFrame(X, columns=col_names)
    df["class"] = y
    df.to_csv(path+f'{dataid}_{dataset.name}.csv', index=False)
    print(dataset.name)

In [4]:
files = os.listdir(path)
scores = {'lr': [], 'knn': [], 'dt': []}
clfs = {'lr':LogisticRegression(random_state=seed),
        'knn': KNeighborsClassifier(),
        'dt': DecisionTreeClassifier(random_state=seed)}
fold = 10
kfold = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)

In [5]:
for file in progress_bar(files):
    df = pd.read_csv(path+file)
    X = df.drop(columns=["class"]).values
    y = df["class"].values
    for clf in clfs.items():
        score = []
        for train_idx, test_idx in kfold.split(X, y):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test, y_test = X[test_idx], y[test_idx]
            clf[1].fit(X_train, y_train)
            score.append(100.*clf[1].score(X_test, y_test))
        scores[clf[0]].append(np.mean(score))

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

...

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jader/Projects/deep-meta-learning/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

In [9]:
files = os.listdir(path)
ratio = np.random.rand(len(files))
df = pd.DataFrame({'files': files, 'train': ratio > .1, **scores})
df.to_csv("../data.csv", index=False)

In [10]:
df.head()

,files,train,lr,knn,dt
0,888_fri_c0_500_50.csv,True,81.600000,69.000000,73.400000
1,37_diabetes.csv,True,77.467532,73.426179,70.423787
2,785_wind_correlations.csv,True,86.500000,84.500000,89.000000
3,40702_solar-flare.csv,True,83.390055,82.831070,80.859637
4,885_transplant.csv,True,97.692308,96.978022,98.461538
